# Please find below the high-level structuring of python scripts in the attached folder. 

## Feature Engineering Script: 
* Feature Engineering_PPM  --- PPM, PRNR, Premium Freight

## Individual Model Scripts:
* regression approach PPM-v3
* regression approach prem freight 
* regression approach prnr
* Vulnerability Score Reg ----contains the feature engineering piece of vulnerability score as well

## Prod Scripts:
* Prod Script_V1   --- PPM, PRNR, Premium Freight
* Prod Script Vulnerability Score  ----- Vulnerability Score

## We need to run the prod scripts in the 1st week of every month

In [1]:
import pandas as pd
import numpy as np
import os
from dateutil.relativedelta import relativedelta
from datetime import date, datetime
import xgboost as xgb
import pickle
import cx_Oracle
os.chdir(r"C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\Supplier_Risk")

### Import Data Files

In [2]:
# tns_dsn = cx_Oracle.makedsn('aamlxbidbp001.aam.net', 1525, service_name='BIDWPRD')
# con_ORACLE = cx_Oracle.connect(user='PROD_AAM_RO', password='prodaamro', dsn=tns_dsn)

# df = pd.read_sql_query("""select * from bi_dw.WC_SC_SUMMARY_V
#                        where month >= 201910
#                        """, con_ORACLE)

In [3]:
# df.to_csv('data/prod_script_v1_data.csv')

In [4]:
df = pd.read_csv('data/prod_script_v1_data.csv')

In [5]:
df.tail()

,Unnamed: 0,TYPE,VENDOR_NAME,ORGANIZATION_CODE,MONTH,VAL
207890,207890,PPM Qty Rejected,DAIMLER INDIA COMMERCIAL VEHICLES PRIVATE LIMITED,PCD,202106,0.0
207891,207891,PPM Qty Rejected,DAIMLER INDIA COMMERCIAL VEHICLES PRIVATE LIMITED,PCD,202208,0.0
207892,207892,PPM Qty Rejected,DAIMLER INDIA COMMERCIAL VEHICLES PRIVATE LIMITED,PCD,202304,0.0
207893,207893,PPM Qty Rejected,DAIMLER INDIA COMMERCIAL VEHICLES PRIVATE LIMITED,PCD,202305,0.0
207894,207894,PPM Qty Rejected,MADRAS ENGINEERING INDUSTRIES PVT LTD,PCD,202103,0.0


In [6]:
# Download data from Orcale DB for prod
# Logic to collate data and filter suppliers to be implemented in the SQL script itself
# df = pd.read_csv(os.getcwd()+r"\Data Files\Feature Engineering Base.csv", encoding='latin-1')
df1 = pd.read_csv(os.getcwd()+r"\data\Mapping File Prod.csv", encoding='latin-1')

In [7]:
df = df.merge(df1, on = ["VENDOR_NAME","ORGANIZATION_CODE"], how='left')

In [8]:
df.head()

,Unnamed: 0,TYPE,VENDOR_NAME,ORGANIZATION_CODE,MONTH,VAL,ORG_TYPE
0,0,Qty Recd,SourcePro EGC de Mexico S de RL de CV,QDI,202205,45298.0,Others2
1,1,Qty Recd,PRODUCTOS ESPECIALIZADOS DE ACERO SA DE CV,LDD,202108,129697.0,Others
2,2,Qty Recd,ANVI MEXICANA SA DE CV,QDI,202209,6.0,Others2
3,3,Qty Recd,ALUDYNE AUTOMOTIVE MEXICO CITY S A DE C V,LDD,202207,157120.0,Others
4,4,Qty Recd,SHANGHAI JANWEI PACKING MATERIAL CO LTD,CDI,202202,719.0,Others2


In [9]:
# df = df[df["ORG_TYPE"]!="Others2"]
df = df.groupby(['VENDOR_NAME', 'MONTH', 'TYPE']).sum().reset_index()

In [10]:
df = df.pivot(index = ['VENDOR_NAME','MONTH'], columns=['TYPE'], values = 'VAL').reset_index()

### Data Cleaning 

In [11]:
df["Qty Recd"].fillna(0, inplace = True)

In [12]:
df["QTY_RECD_FLAG"] = np.where((df["Qty Ordered"] != np.NaN) & (df["Qty Ordered"] !=0), df["Qty Recd"]/df["Qty Ordered"], np.NaN)
df["QTY_RECD_FLAG"] = np.where(df["QTY_RECD_FLAG"]>1, 1, df["QTY_RECD_FLAG"])

In [13]:
df["PPM Flag"] = np.where((df["PPM Qty Shipped"] != np.NaN) & (df["PPM Qty Shipped"] != 0), df["PPM Qty Rejected"]/df["PPM Qty Shipped"]*1000000, np.NaN)

In [14]:
df["MONTH"] = pd.to_datetime(df["MONTH"], format = '%Y%m').dt.date

### Define Current Month

In [15]:
cur_month = date(date.today().year, date.today().month, 1)

cur_month = date(2023,6,1)

In [16]:
cur_month

datetime.date(2023, 6, 1)

In [17]:
df = df[df["MONTH"]>= cur_month - relativedelta(months=12)]

In [18]:
df.tail()

TYPE,VENDOR_NAME,MONTH,Cost Recovery,PPM Qty Rejected,PPM Qty Shipped,PREM FREIGHT,PRNR,Qty Ordered,Qty Recd,QTY_RECD_FLAG,PPM Flag
57436,a+f Poland Sp z o o,2023-02-01,NaN,0.0,6120.0,NaN,NaN,6120.0,9000.0,1.0,0.0
57437,a+f Poland Sp z o o,2023-03-01,NaN,0.0,5760.0,NaN,NaN,5761.0,6480.0,1.0,0.0
57438,a+f Poland Sp z o o,2023-04-01,NaN,0.0,6480.0,NaN,NaN,6483.0,10800.0,1.0,0.0
57439,a+f Poland Sp z o o,2023-05-01,NaN,0.0,1440.0,NaN,NaN,1440.0,1440.0,1.0,0.0
57440,a+f Poland Sp z o o,2023-06-01,NaN,NaN,NaN,NaN,NaN,720.0,0.0,0.0,NaN


### QTY_RECD_FLAG to be aggregated w/o vendor name

In [19]:
target_df = df[["VENDOR_NAME"]]
target_df.drop_duplicates(inplace = True)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
final_target_df = pd.DataFrame()


month = cur_month

target_df = df[["VENDOR_NAME"]]
target_df.drop_duplicates(inplace = True)


#Month on Month features
for z in np.arange(1,13,1):
    tmp = df[df["MONTH"]==month - relativedelta(months=z)].groupby(['VENDOR_NAME']).agg({ 
                "QTY_RECD_FLAG" : ['mean'],
                "Qty Ordered":['sum'], 
                "Qty Recd":['sum'],
                "PRNR" : ['sum'],
                "PREM FREIGHT" : ['sum'],
                "Cost Recovery" : ['sum'],
                "PPM Qty Rejected": ['sum'],
                "PPM Qty Shipped":['sum']
                })
    tmp.columns = [s + "_MOM_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
    tmp.reset_index(inplace = True)
    target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')


#Quarterly features
for z in np.arange(1,13,3):
    tmp = df[(df["MONTH"]>=month - relativedelta(months=z+2))&(df["MONTH"]<=month - relativedelta(months=z))].groupby(['VENDOR_NAME']).agg({ 
                "QTY_RECD_FLAG" : ['max','min','std','mean'],
                "Qty Ordered":['max','min','std','mean', 'sum'], 
                "Qty Recd":['max','min','std','mean','sum'],
                "PRNR" : ['max','std','mean','sum'],
                "PREM FREIGHT" : ['max','std','mean','sum'],
                "Cost Recovery" : ['max','std','mean','sum'],
                "PPM Flag" : ['max','min','std','mean'], 
                "PPM Qty Rejected": ['max','min','std','mean', 'sum'],
                "PPM Qty Shipped":['max','min','std','mean', 'sum']
                })
    tmp.columns = [s + "_OQT_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
    tmp.reset_index(inplace = True)
    
    for z in [x for x in tmp.columns if "PPM Qty Shipped_sum" in x]:
        tmp[z.replace("PPM Qty Shipped_sum","PPM_Agg_Flag")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), np.round(tmp[z.replace("Shipped","Rejected")]/tmp[z]*1000000), np.NaN)
        
    for z in [x for x in tmp.columns if "Qty Ordered_sum" in x]:
        tmp[z.replace("Qty Ordered_sum","Qty Recd_percent_Agg")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), tmp[z.replace("Ordered","Recd")]/tmp[z]*100, np.NaN) 


    target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')


final_target_df = pd.concat([final_target_df, target_df])

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
filename ='data/PRNR_selected_cols.sav'
selected_vars_prnr = pickle.load(open(filename, 'rb'))
selected_vars_prnr

['Qty Recd_sum_MOM_1',
 'Qty Recd_max_OQT_1',
 'PRNR_mean_OQT_1',
 'Qty Recd_sum_OQT_1',
 'Qty Recd_mean_OQT_1',
 'PPM Qty Shipped_mean_OQT_7',
 'PPM Qty Shipped_sum_OQT_4',
 'PRNR_max_OQT_1',
 'PPM Qty Shipped_mean_OQT_1']

In [22]:
# selected_vars_prnr = ['Qty Recd_sum_MOM_1',
#   'Qty Recd_max_OQT_1',
#   'Qty Recd_sum_OQT_1',
#   'Qty Recd_mean_OQT_1',
#   'PRNR_max_OQT_1',
#   'PRNR_mean_OQT_4',
#   'PRNR_mean_OQT_1',
#   'PPM Qty Shipped_min_OQT_4',
#   'PPM Qty Shipped_max_OQT_7']

In [23]:
bst = pickle.load(open("xgb_prnr_reg", "rb"))

In [24]:
final_target_df["PRED_PRNR"] =  bst.predict(final_target_df[selected_vars_prnr], ntree_limit=bst.best_ntree_limit)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\xgboost\core.py:108: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


In [25]:
final_target_df.head()

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Rejected_mean_OQT_10,PPM Qty Rejected_sum_OQT_10,PPM Qty Shipped_max_OQT_10,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR
0,*AAM INTERNATIONAL S.A.R.L. (LUXCO DRIVELINE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.040974
1,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.067772
2,*CERATIZIT CHICAGO INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.040974
3,*DRILL COMPANY DE MEXICO SA DE CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.040974
4,*HENNIGES AUTOMOTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040974


In [26]:
final_target_df.shape

(2717, 266)

In [27]:
list(final_target_df)

['VENDOR_NAME',
 'QTY_RECD_FLAG_mean_MOM_1',
 'Qty Ordered_sum_MOM_1',
 'Qty Recd_sum_MOM_1',
 'PRNR_sum_MOM_1',
 'PREM FREIGHT_sum_MOM_1',
 'Cost Recovery_sum_MOM_1',
 'PPM Qty Rejected_sum_MOM_1',
 'PPM Qty Shipped_sum_MOM_1',
 'QTY_RECD_FLAG_mean_MOM_2',
 'Qty Ordered_sum_MOM_2',
 'Qty Recd_sum_MOM_2',
 'PRNR_sum_MOM_2',
 'PREM FREIGHT_sum_MOM_2',
 'Cost Recovery_sum_MOM_2',
 'PPM Qty Rejected_sum_MOM_2',
 'PPM Qty Shipped_sum_MOM_2',
 'QTY_RECD_FLAG_mean_MOM_3',
 'Qty Ordered_sum_MOM_3',
 'Qty Recd_sum_MOM_3',
 'PRNR_sum_MOM_3',
 'PREM FREIGHT_sum_MOM_3',
 'Cost Recovery_sum_MOM_3',
 'PPM Qty Rejected_sum_MOM_3',
 'PPM Qty Shipped_sum_MOM_3',
 'QTY_RECD_FLAG_mean_MOM_4',
 'Qty Ordered_sum_MOM_4',
 'Qty Recd_sum_MOM_4',
 'PRNR_sum_MOM_4',
 'PREM FREIGHT_sum_MOM_4',
 'Cost Recovery_sum_MOM_4',
 'PPM Qty Rejected_sum_MOM_4',
 'PPM Qty Shipped_sum_MOM_4',
 'QTY_RECD_FLAG_mean_MOM_5',
 'Qty Ordered_sum_MOM_5',
 'Qty Recd_sum_MOM_5',
 'PRNR_sum_MOM_5',
 'PREM FREIGHT_sum_MOM_5',
 'Cost R

In [28]:
# load selected_vars_prem

In [29]:
filename ='data/prem_freight_selected_cols.sav'
prem_freight_selected_cols = pickle.load(open(filename, 'rb'))

In [30]:
selected_vars_prem = prem_freight_selected_cols
# selected_vars_prem = ['Qty Ordered_sum_MOM_11',
#   'PREM FREIGHT_sum_MOM_1',
#   'Qty Recd_sum_MOM_10',
#   'PREM FREIGHT_sum_OQT_1',
#   'PPM Flag_max_OQT_7',
#   'PREM FREIGHT_max_OQT_1',
#   'PPM Qty Shipped_sum_MOM_6',
#   'Qty Ordered_sum_MOM_10',
#   'PREM FREIGHT_mean_OQT_1',
#   'Qty Ordered_min_OQT_4']
selected_vars_prem

['PREM FREIGHT_std_OQT_1',
 'PREM FREIGHT_sum_MOM_1',
 'PREM FREIGHT_mean_OQT_1',
 'PREM FREIGHT_sum_OQT_1',
 'PREM FREIGHT_sum_MOM_2',
 'PREM FREIGHT_max_OQT_1',
 'PREM FREIGHT_sum_MOM_3',
 'PREM FREIGHT_sum_MOM_10',
 'PREM FREIGHT_sum_MOM_9',
 'PREM FREIGHT_max_OQT_4',
 'PREM FREIGHT_sum_MOM_6',
 'PREM FREIGHT_sum_MOM_5']

In [31]:
bst = pickle.load(open("xgb_prem_freight_reg", "rb"))

In [32]:
# final_target_df['PPM Flag'] = 0
final_target_df["PRED_PREM_FREIGHT"] =  bst.predict(final_target_df[selected_vars_prem], ntree_limit=bst.best_ntree_limit)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\xgboost\core.py:108: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


In [33]:
filename ='data/PPM_selected_cols.sav'
selected_vars_ppm = pickle.load(open(filename, 'rb'))
selected_vars_ppm

['PPM Flag_mean_OQT_1',
 'PPM_Agg_Flag_OQT_1',
 'PPM Flag_max_OQT_1',
 'PPM Qty Rejected_sum_OQT_1',
 'PPM Qty Rejected_mean_OQT_1',
 'PPM Qty Rejected_min_OQT_1',
 'PPM Flag_std_OQT_1',
 'PPM Flag_min_OQT_1',
 'PRNR_sum_OQT_1',
 'PPM Qty Rejected_mean_OQT_4',
 'PPM Qty Rejected_sum_MOM_1',
 'Cost Recovery_sum_OQT_1',
 'PPM Flag_mean_OQT_7',
 'Cost Recovery_sum_OQT_7',
 'PPM Qty Rejected_sum_OQT_4',
 'PPM Qty Rejected_sum_MOM_2',
 'Cost Recovery_std_OQT_7',
 'PPM Qty Rejected_mean_OQT_10',
 'Cost Recovery_sum_MOM_8',
 'PRNR_max_OQT_1',
 'PPM Qty Rejected_sum_OQT_10',
 'PPM Qty Shipped_mean_OQT_4']

In [34]:
# selected_vars_ppm = ['PPM Flag_mean_OQT_1',
#   'PPM Flag_max_OQT_1',
#   'PPM Qty Rejected_sum_OQT_1',
#   'PPM_Agg_Flag_OQT_1',
#   'PPM Qty Rejected_mean_OQT_1',
#   'PPM Qty Rejected_min_OQT_1',
#   'PRNR_sum_OQT_1',
#   'PPM Flag_min_OQT_1',
#   'PPM Qty Rejected_mean_OQT_4',
#   'PPM Qty Rejected_sum_MOM_1',
#   'PPM Flag_std_OQT_1',
#   'Cost Recovery_sum_OQT_7',
#   'PPM Qty Rejected_sum_MOM_2',
#   'PPM Qty Rejected_mean_OQT_10',
#   'Cost Recovery_sum_OQT_1',
#   'PPM Flag_mean_OQT_7',
#   'PPM Qty Rejected_sum_OQT_4',
#   'PRNR_max_OQT_1',
#   'PPM Qty Shipped_mean_OQT_4',
#   'PPM_Agg_Flag_OQT_4',
#   'PPM Qty Shipped_max_OQT_4']

In [35]:
bst = pickle.load(open("xgb_ppm_reg_v3", "rb"))

In [36]:
final_target_df["PRED_PPM"] =  np.power(10,bst.predict(final_target_df[selected_vars_ppm], ntree_limit=bst.best_ntree_limit))*100
final_target_df["PRED_PPM"] = np.where(final_target_df["PRED_PPM"]<200,0,final_target_df["PRED_PPM"])

In [37]:
# final_target_df["FLAG_PREM_FREIGHT"] =  np.where(final_target_df["PRED_PREM_FREIGHT"]>=2.5,3,np.where(final_target_df["PRED_PREM_FREIGHT"]>=1.75,2,np.where(final_target_df["PRED_PREM_FREIGHT"]>=1,1,0)))
# final_target_df["FLAG_PRNR"]=np.where(final_target_df["PRED_PRNR"]>=1.5,3,np.where(final_target_df["PRED_PRNR"]>=1,2,np.where(final_target_df["PRED_PRNR"]>=0.75,1,0)))
# final_target_df["FLAG_PPM"]=np.where(final_target_df["PRED_PPM"]>=450,3,np.where(final_target_df["PRED_PPM"]>=290,2,np.where(final_target_df["PRED_PPM"]>=200,1,0)))

In [38]:
final_target_df["MONTH"] = cur_month

In [39]:
final_target_df.head(5)

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PREM_FREIGHT,PRED_PPM,MONTH
0,*AAM INTERNATIONAL S.A.R.L. (LUXCO DRIVELINE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,0.040974,0.012318,0.0,2023-06-01
1,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.067772,0.013388,0.0,2023-06-01
2,*CERATIZIT CHICAGO INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01
3,*DRILL COMPANY DE MEXICO SA DE CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01
4,*HENNIGES AUTOMOTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.040974,0.012410,0.0,2023-06-01


In [40]:
final_target_df.sort_values('PRED_PREM_FREIGHT', ascending=False)

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PREM_FREIGHT,PRED_PPM,MONTH
1107,INDUSTRIAL MOLDING CORP,0.819461,678700.0,556168.0,1.0,2.0,0.0,0.0,479168.0,1.000000,...,855183.0,48971.686966,909035.666667,2727107.0,593.0,100.834387,0.717198,4.977916,335.591309,2023-06-01
706,ESSENTRA COMPONENTS,0.835254,368750.0,308000.0,2.0,2.0,0.0,0.0,273750.0,0.869775,...,222749.0,100534.952569,334493.000000,1003479.0,0.0,101.402000,0.910404,4.800596,0.000000,2023-06-01
2015,SAB MONTMERLE,0.842023,132861.0,111872.0,1.0,2.0,1.0,0.0,115836.0,0.849273,...,58789.0,19151.414578,77079.333333,231238.0,0.0,95.873891,0.965853,4.800596,347.177429,2023-06-01
929,GREENDALE SCREW PRODUCTS INC.,0.632485,396000.0,250464.0,1.0,5.0,6.0,1.0,299180.0,0.924951,...,286601.0,30658.268613,318634.333333,955903.0,0.0,103.846412,0.715205,3.073239,299.123230,2023-06-01
6,*MENNIE MACHINE COMPANY,NaN,0.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.115889,2.855810,0.000000,2023-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393,LIGHT HOUSE ENGINEERING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.029798,0.009747,0.000000,2023-06-01
688,ENVISION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.029798,0.009747,0.000000,2023-06-01
392,CENCI E CIA LTDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.029798,0.009747,0.000000,2023-06-01
573,DIVERSIFIED MANUFACTURING AND ASSEMBLY LLC,1.000000,4197.0,4210.0,0.0,0.0,0.0,0.0,3560.0,1.000000,...,3090.0,1681.706673,4518.666667,13556.0,0.0,92.586114,0.199765,-0.002024,0.000000,2023-06-01


# Output 

### Save Output to CSV 

In [41]:
# final_target_df[["VENDOR_NAME","MONTH","PRED_PRNR","PRED_PREM_FREIGHT","FLAG_PREM_FREIGHT","FLAG_PRNR", "PRED_PPM", "FLAG_PPM"]].to_csv(os.getcwd()+r"\data\output_file_prod.csv", index=False)

In [42]:
prv_month = date(date.today().year, date.today().month, 1)
prv_month = prv_month - relativedelta(months=1)
prv_month

datetime.date(2023, 5, 1)

In [43]:
df_pred = df[df["MONTH"] == prv_month]
df_pred

TYPE,VENDOR_NAME,MONTH,Cost Recovery,PPM Qty Rejected,PPM Qty Shipped,PREM FREIGHT,PRNR,Qty Ordered,Qty Recd,QTY_RECD_FLAG,PPM Flag
320,*MENNIE MACHINE COMPANY,2023-05-01,NaN,NaN,NaN,2.0,NaN,NaN,0.0,NaN,NaN
516,2A MATERIAIS ELETRICOS LTDA,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN
542,3 DIMENSIONAL SERVICES,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
587,A & F GROBSERIEN GMBH,2023-05-01,NaN,0.0,14304.0,NaN,NaN,31645.0,29304.0,0.926023,0.0
620,A N PRECISION CO LTD,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
57266,ZOLLER INC,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN
57325,ZOUPING TIANSHENG METAL TECHNOLOGY CO LTD,2023-05-01,NaN,0.0,18744.0,NaN,NaN,20976.0,20904.0,0.996568,0.0
57374,ZWZ BEARING USA,2023-05-01,NaN,0.0,174240.0,NaN,NaN,222144.0,155808.0,0.701383,0.0
57396,ZWZ Bearing Europe Gmbh,2023-05-01,NaN,0.0,3456.0,NaN,NaN,3456.0,3456.0,1.000000,0.0


In [44]:
# tns_dsn = cx_Oracle.makedsn('aamlxbidbp001.aam.net', 1525, service_name='BIDWPRD')
# con_ORACLE = cx_Oracle.connect(user='PROD_AAM_RO', password='prodaamro', dsn=tns_dsn)

In [45]:
# df_pred = pd.read_sql_query("""select * from bi_dw.wc_sc_pred_s
#                        where MONTH = '01-OCT-22'
#                        """, con_ORACLE)

In [46]:
df_pred.head()

TYPE,VENDOR_NAME,MONTH,Cost Recovery,PPM Qty Rejected,PPM Qty Shipped,PREM FREIGHT,PRNR,Qty Ordered,Qty Recd,QTY_RECD_FLAG,PPM Flag
320,*MENNIE MACHINE COMPANY,2023-05-01,NaN,NaN,NaN,2.0,NaN,NaN,0.0,NaN,NaN
516,2A MATERIAIS ELETRICOS LTDA,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN
542,3 DIMENSIONAL SERVICES,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN
587,A & F GROBSERIEN GMBH,2023-05-01,NaN,0.0,14304.0,NaN,NaN,31645.0,29304.0,0.926023,0.0
620,A N PRECISION CO LTD,2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN


In [47]:
def is_what_percent_of(num_a, num_b):
    return (num_a / 100) * num_b

In [48]:
increase1 = []
for index, row in final_target_df.iterrows():
    this_increase = is_what_percent_of(row['PRED_PREM_FREIGHT'],10)
#     print(this_increase)
    increase1.append(this_increase)
    
final_target_df['PRED_PREM_FREIGHT_inc1'] = increase1

In [49]:
final_target_df.shape

(2717, 270)

In [50]:
increase = []
for index, row in df_pred.iterrows():
    this_increase = row['PREM FREIGHT'] -is_what_percent_of(row['PREM FREIGHT'],10)
    increase.append(this_increase)
    
df_pred['PRED_PREM_FREIGHT_inc'] = increase

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [51]:
df_pred.shape

(1596, 12)

In [52]:
df_pred1 = df_pred[['VENDOR_NAME','PRED_PREM_FREIGHT_inc']]

In [53]:
df_pred1

TYPE,VENDOR_NAME,PRED_PREM_FREIGHT_inc
320,*MENNIE MACHINE COMPANY,1.8
516,2A MATERIAIS ELETRICOS LTDA,NaN
542,3 DIMENSIONAL SERVICES,NaN
587,A & F GROBSERIEN GMBH,NaN
620,A N PRECISION CO LTD,NaN
...,...,...
57266,ZOLLER INC,NaN
57325,ZOUPING TIANSHENG METAL TECHNOLOGY CO LTD,NaN
57374,ZWZ BEARING USA,NaN
57396,ZWZ Bearing Europe Gmbh,NaN


In [54]:
df_pred1["PRED_PREM_FREIGHT_inc"].fillna(0, inplace = True)
df_pred1

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


TYPE,VENDOR_NAME,PRED_PREM_FREIGHT_inc
320,*MENNIE MACHINE COMPANY,1.8
516,2A MATERIAIS ELETRICOS LTDA,0.0
542,3 DIMENSIONAL SERVICES,0.0
587,A & F GROBSERIEN GMBH,0.0
620,A N PRECISION CO LTD,0.0
...,...,...
57266,ZOLLER INC,0.0
57325,ZOUPING TIANSHENG METAL TECHNOLOGY CO LTD,0.0
57374,ZWZ BEARING USA,0.0
57396,ZWZ Bearing Europe Gmbh,0.0


In [55]:
df_pred1['PRED_PREM_FREIGHT_inc'] = df_pred1['PRED_PREM_FREIGHT_inc'].astype(int)
df_pred1['VENDOR_NAME'] = df_pred1['VENDOR_NAME'].astype(str)


C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
final_target_df['PRED_PREM_FREIGHT_inc1'] = final_target_df['PRED_PREM_FREIGHT_inc1'].astype(int)
final_target_df['VENDOR_NAME'] = final_target_df['VENDOR_NAME'].astype(str)


In [57]:
# result = pd.merge(final_target_df, df_pred, left_on="VENDOR_NAME", right_index=True, how="left", sort=False)

In [58]:
result = pd.merge(
    final_target_df,
    df_pred1,
    how="left",
    left_on='VENDOR_NAME',
#     left_index=True,
    right_on='VENDOR_NAME')

In [59]:
result

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PREM_FREIGHT,PRED_PPM,MONTH,PRED_PREM_FREIGHT_inc1,PRED_PREM_FREIGHT_inc
0,*AAM INTERNATIONAL S.A.R.L. (LUXCO DRIVELINE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.000000,0.040974,0.012318,0.0,2023-06-01,0,NaN
1,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.067772,0.013388,0.0,2023-06-01,0,NaN
2,*CERATIZIT CHICAGO INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN
3,*DRILL COMPANY DE MEXICO SA DE CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN
4,*HENNIGES AUTOMOTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,ZWEZ-CHEMIE GMBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.040974,0.012318,0.0,2023-06-01,0,NaN
2713,ZWZ BEARING USA,0.701383,222144.0,155808.0,0.0,0.0,0.0,0.0,174240.0,0.974194,...,165264.0,495792.0,0.0,102.826855,0.348462,0.014627,0.0,2023-06-01,0,0.0
2714,ZWZ Bearing Europe Gmbh,1.000000,3456.0,3456.0,0.0,0.0,0.0,0.0,3456.0,NaN,...,4608.0,13824.0,0.0,100.000000,0.209499,0.012943,0.0,2023-06-01,0,0.0
2715,Zoller Tecnologias S de RL de CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.022977,0.011821,0.0,2023-06-01,0,NaN


In [60]:
final_prm_fre = []
for index, row in result.iterrows():
    if row['PRED_PREM_FREIGHT_inc'] == 0:
        final_prm_fre.append(row['PRED_PREM_FREIGHT'])
    else:
        final_prm_fre.append(row['PRED_PREM_FREIGHT_inc1']+row['PRED_PREM_FREIGHT_inc'])
result['PRED_PREM_FREIGHT_final'] = final_prm_fre

In [61]:
result

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PREM_FREIGHT,PRED_PPM,MONTH,PRED_PREM_FREIGHT_inc1,PRED_PREM_FREIGHT_inc,PRED_PREM_FREIGHT_final
0,*AAM INTERNATIONAL S.A.R.L. (LUXCO DRIVELINE),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.000000,0.040974,0.012318,0.0,2023-06-01,0,NaN,NaN
1,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.067772,0.013388,0.0,2023-06-01,0,NaN,NaN
2,*CERATIZIT CHICAGO INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN,NaN
3,*DRILL COMPANY DE MEXICO SA DE CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN,NaN
4,*HENNIGES AUTOMOTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.040974,0.012410,0.0,2023-06-01,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2712,ZWEZ-CHEMIE GMBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.040974,0.012318,0.0,2023-06-01,0,NaN,NaN
2713,ZWZ BEARING USA,0.701383,222144.0,155808.0,0.0,0.0,0.0,0.0,174240.0,0.974194,...,495792.0,0.0,102.826855,0.348462,0.014627,0.0,2023-06-01,0,0.0,0.014627
2714,ZWZ Bearing Europe Gmbh,1.000000,3456.0,3456.0,0.0,0.0,0.0,0.0,3456.0,NaN,...,13824.0,0.0,100.000000,0.209499,0.012943,0.0,2023-06-01,0,0.0,0.012943
2715,Zoller Tecnologias S de RL de CV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.022977,0.011821,0.0,2023-06-01,0,NaN,NaN


In [62]:
ll = result.sort_values('PRED_PREM_FREIGHT_final', ascending=False)

In [63]:
ll2 = ll.drop(['PRED_PREM_FREIGHT_inc1','PRED_PREM_FREIGHT_inc','PRED_PREM_FREIGHT'], axis=1)
ll2.head()

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PPM,MONTH,PRED_PREM_FREIGHT_final
1557,"METALDYNE SINTERED RIDGWAY, LLC",0.888234,78906.0,70087.0,0.0,7.0,5.0,0.0,71087.0,0.953997,...,75228.0,7656.708322,83049.666667,249149.0,8.0,98.412191,0.322917,287.915070,2023-06-01,6.000000
929,GREENDALE SCREW PRODUCTS INC.,0.632485,396000.0,250464.0,1.0,5.0,6.0,1.0,299180.0,0.924951,...,286601.0,30658.268613,318634.333333,955903.0,0.0,103.846412,0.715205,299.123230,2023-06-01,4.000000
868,GENERAL PLUG & MANUFACTURING,0.653540,728800.0,476300.0,0.0,0.0,0.0,0.0,543400.0,0.861374,...,477505.0,84332.717058,574017.333333,1722052.0,0.0,102.371922,0.730068,272.353699,2023-06-01,2.473634
1011,HHI FORMTECH LLC,1.000000,243102.0,246841.0,0.0,0.0,0.0,0.0,268065.0,1.000000,...,231993.0,27678.313261,262513.000000,787539.0,0.0,122.930777,0.408171,0.000000,2023-06-01,1.593688
1270,KELSEY HAYES CO.,0.719929,122044.0,87863.0,0.0,0.0,0.0,0.0,91513.0,1.000000,...,93809.0,2834.875012,95871.333333,287614.0,3.0,107.949789,1.434842,519.622009,2023-06-01,1.113197


In [64]:
ll2

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PPM,MONTH,PRED_PREM_FREIGHT_final
1557,"METALDYNE SINTERED RIDGWAY, LLC",0.888234,78906.0,70087.0,0.0,7.0,5.0,0.0,71087.0,0.953997,...,75228.0,7656.708322,83049.666667,249149.0,8.0,98.412191,0.322917,287.915070,2023-06-01,6.000000
929,GREENDALE SCREW PRODUCTS INC.,0.632485,396000.0,250464.0,1.0,5.0,6.0,1.0,299180.0,0.924951,...,286601.0,30658.268613,318634.333333,955903.0,0.0,103.846412,0.715205,299.123230,2023-06-01,4.000000
868,GENERAL PLUG & MANUFACTURING,0.653540,728800.0,476300.0,0.0,0.0,0.0,0.0,543400.0,0.861374,...,477505.0,84332.717058,574017.333333,1722052.0,0.0,102.371922,0.730068,272.353699,2023-06-01,2.473634
1011,HHI FORMTECH LLC,1.000000,243102.0,246841.0,0.0,0.0,0.0,0.0,268065.0,1.000000,...,231993.0,27678.313261,262513.000000,787539.0,0.0,122.930777,0.408171,0.000000,2023-06-01,1.593688
1270,KELSEY HAYES CO.,0.719929,122044.0,87863.0,0.0,0.0,0.0,0.0,91513.0,1.000000,...,93809.0,2834.875012,95871.333333,287614.0,3.0,107.949789,1.434842,519.622009,2023-06-01,1.113197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,ZOBOR INDUSTRIA MECANICA LTDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.029798,0.000000,2023-06-01,NaN
2709,ZOLLER SHANGHAI LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.022977,0.000000,2023-06-01,NaN
2711,ZUI SHO PRECISION IND. CO.LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.000000,0.0,NaN,NaN,0.040974,0.000000,2023-06-01,NaN
2712,ZWEZ-CHEMIE GMBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.040974,0.000000,2023-06-01,NaN


In [65]:
ll2.rename(columns = {'PRED_PREM_FREIGHT_final':'PRED_PREM_FREIGHT'}, inplace = True)
ll2.head()

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PPM,MONTH,PRED_PREM_FREIGHT
1557,"METALDYNE SINTERED RIDGWAY, LLC",0.888234,78906.0,70087.0,0.0,7.0,5.0,0.0,71087.0,0.953997,...,75228.0,7656.708322,83049.666667,249149.0,8.0,98.412191,0.322917,287.915070,2023-06-01,6.000000
929,GREENDALE SCREW PRODUCTS INC.,0.632485,396000.0,250464.0,1.0,5.0,6.0,1.0,299180.0,0.924951,...,286601.0,30658.268613,318634.333333,955903.0,0.0,103.846412,0.715205,299.123230,2023-06-01,4.000000
868,GENERAL PLUG & MANUFACTURING,0.653540,728800.0,476300.0,0.0,0.0,0.0,0.0,543400.0,0.861374,...,477505.0,84332.717058,574017.333333,1722052.0,0.0,102.371922,0.730068,272.353699,2023-06-01,2.473634
1011,HHI FORMTECH LLC,1.000000,243102.0,246841.0,0.0,0.0,0.0,0.0,268065.0,1.000000,...,231993.0,27678.313261,262513.000000,787539.0,0.0,122.930777,0.408171,0.000000,2023-06-01,1.593688
1270,KELSEY HAYES CO.,0.719929,122044.0,87863.0,0.0,0.0,0.0,0.0,91513.0,1.000000,...,93809.0,2834.875012,95871.333333,287614.0,3.0,107.949789,1.434842,519.622009,2023-06-01,1.113197


In [66]:
# ll2["PRED_PREM_FREIGHT"] = ll2["PRED_PREM_FREIGHT"].round()
# ll2["PRED_PRNR"] = ll2["PRED_PRNR"].round()
# ll2["PRED_PPM"] = ll2["PRED_PPM"].round()

In [67]:
# ll2["FLAG_PREM_FREIGHT"] =  np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=15,3,np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=11,2,np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=6,1,0)))


In [68]:
# ll2["FLAG_PREM_FREIGHT"] =  np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=15,3,np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=11,2,np.where(((ll2["PRED_PREM_FREIGHT"].round())*6)>=6,1,0)))
# ll2["FLAG_PRNR"]=np.where(((ll2["PRED_PRNR"].round())*6)>=9,3,np.where(((ll2["PRED_PRNR"].round())*6)>=6,2,np.where(((ll2["PRED_PRNR"].round())*6)>=4,1,0)))
# ll2["FLAG_PPM"]=np.where((ll2["PRED_PPM"].round())>=450,3,np.where((ll2["PRED_PPM"].round())>=290,2,np.where((ll2["PRED_PPM"].round())>=200,1,0)))

In [98]:
ll2["FLAG_PREM_FREIGHT"] =  np.where(((ll2["PRED_PREM_FREIGHT"]*6).round())>=15,3,np.where(((ll2["PRED_PREM_FREIGHT"]*6).round())>=11,2,np.where(((ll2["PRED_PREM_FREIGHT"]*6).round())>=6,1,0)))
ll2["FLAG_PRNR"]=np.where(((ll2["PRED_PRNR"]*6).round())>=9,3,np.where(((ll2["PRED_PRNR"]*6).round())>=6,2,np.where(((ll2["PRED_PRNR"]*6).round())>=4,1,0)))
ll2["FLAG_PPM"]=np.where((ll2["PRED_PPM"]*6)>=450,3,np.where((ll2["PRED_PPM"]*6)>=290,2,np.where((ll2["PRED_PPM"]*6)>=200,1,0)))

In [99]:
ll2

,VENDOR_NAME,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,Cost Recovery_sum_MOM_1,PPM Qty Rejected_sum_MOM_1,PPM Qty Shipped_sum_MOM_1,QTY_RECD_FLAG_mean_MOM_2,...,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10,PRED_PRNR,PRED_PPM,MONTH,PRED_PREM_FREIGHT,FLAG_PREM_FREIGHT,FLAG_PRNR,FLAG_PPM
1557,"METALDYNE SINTERED RIDGWAY, LLC",0.888234,78906.0,70087.0,0.0,7.0,5.0,0.0,71087.0,0.953997,...,249149.0,8.0,98.412191,0.322917,287.915070,2023-06-01,6.000000,3,0,3
929,GREENDALE SCREW PRODUCTS INC.,0.632485,396000.0,250464.0,1.0,5.0,6.0,1.0,299180.0,0.924951,...,955903.0,0.0,103.846412,0.715205,299.123230,2023-06-01,4.000000,3,1,3
868,GENERAL PLUG & MANUFACTURING,0.653540,728800.0,476300.0,0.0,0.0,0.0,0.0,543400.0,0.861374,...,1722052.0,0.0,102.371922,0.730068,272.353699,2023-06-01,2.473634,3,1,3
1011,HHI FORMTECH LLC,1.000000,243102.0,246841.0,0.0,0.0,0.0,0.0,268065.0,1.000000,...,787539.0,0.0,122.930777,0.408171,0.000000,2023-06-01,1.593688,1,0,0
1270,KELSEY HAYES CO.,0.719929,122044.0,87863.0,0.0,0.0,0.0,0.0,91513.0,1.000000,...,287614.0,3.0,107.949789,1.434842,519.622009,2023-06-01,1.113197,1,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,ZOBOR INDUSTRIA MECANICA LTDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.029798,0.000000,2023-06-01,NaN,0,0,0
2709,ZOLLER SHANGHAI LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.022977,0.000000,2023-06-01,NaN,0,0,0
2711,ZUI SHO PRECISION IND. CO.LTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.040974,0.000000,2023-06-01,NaN,0,0,0
2712,ZWEZ-CHEMIE GMBH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0.040974,0.000000,2023-06-01,NaN,0,0,0


In [100]:
upload_df = ll2[["VENDOR_NAME","MONTH","PRED_PRNR","PRED_PREM_FREIGHT","FLAG_PREM_FREIGHT","FLAG_PRNR","PRED_PPM","FLAG_PPM"]]


In [101]:
upload_df

,VENDOR_NAME,MONTH,PRED_PRNR,PRED_PREM_FREIGHT,FLAG_PREM_FREIGHT,FLAG_PRNR,PRED_PPM,FLAG_PPM
1557,"METALDYNE SINTERED RIDGWAY, LLC",2023-06-01,0.322917,6.000000,3,0,287.915070,3
929,GREENDALE SCREW PRODUCTS INC.,2023-06-01,0.715205,4.000000,3,1,299.123230,3
868,GENERAL PLUG & MANUFACTURING,2023-06-01,0.730068,2.473634,3,1,272.353699,3
1011,HHI FORMTECH LLC,2023-06-01,0.408171,1.593688,1,0,0.000000,0
1270,KELSEY HAYES CO.,2023-06-01,1.434842,1.113197,1,3,519.622009,3
...,...,...,...,...,...,...,...,...
2707,ZOBOR INDUSTRIA MECANICA LTDA,2023-06-01,0.029798,NaN,0,0,0.000000,0
2709,ZOLLER SHANGHAI LTD,2023-06-01,0.022977,NaN,0,0,0.000000,0
2711,ZUI SHO PRECISION IND. CO.LTD,2023-06-01,0.040974,NaN,0,0,0.000000,0
2712,ZWEZ-CHEMIE GMBH,2023-06-01,0.040974,NaN,0,0,0.000000,0


In [102]:
upload_df['PPM'] = upload_df['FLAG_PPM']*6
upload_df['PREM_FREIGHT'] = upload_df['FLAG_PREM_FREIGHT']*6
upload_df['PRNR'] = upload_df['FLAG_PRNR']*6

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:3: SettingW

In [103]:
upload_df.head(50)

,VENDOR_NAME,MONTH,PRED_PRNR,PRED_PREM_FREIGHT,FLAG_PREM_FREIGHT,FLAG_PRNR,PRED_PPM,FLAG_PPM,PPM,PREM_FREIGHT,PRNR
1557,"METALDYNE SINTERED RIDGWAY, LLC",2023-06-01,0.322917,6.000000,3,0,287.915070,3,18,18,0
929,GREENDALE SCREW PRODUCTS INC.,2023-06-01,0.715205,4.000000,3,1,299.123230,3,18,18,6
868,GENERAL PLUG & MANUFACTURING,2023-06-01,0.730068,2.473634,3,1,272.353699,3,18,18,6
1011,HHI FORMTECH LLC,2023-06-01,0.408171,1.593688,1,0,0.000000,0,0,6,0
1270,KELSEY HAYES CO.,2023-06-01,1.434842,1.113197,1,3,519.622009,3,18,6,18
1860,PRODUCTION SPRING LLC,2023-06-01,0.751597,1.102698,1,1,0.000000,0,0,6,6
144,AMG INDUSTRIES LLC,2023-06-01,0.460582,1.102698,1,0,0.000000,0,0,6,0
815,FREUDENBERG NOK GENERAL PARTNERSHIP,2023-06-01,0.947119,1.102698,1,2,225.280350,3,18,6,12
1682,NELCAST LIMITED,2023-06-01,2.535193,1.102698,1,3,2276.036377,3,18,6,18
1107,INDUSTRIAL MOLDING CORP,2023-06-01,0.717198,1.000000,1,1,335.591309,3,18,6,6


In [105]:
upload_df[['VENDOR_NAME','PRED_PPM']].sort_values('PRED_PPM', ascending=False).head(15)

,VENDOR_NAME,PRED_PPM
1497,MARTINREA HONSEL BRASIL FUNDICAO E COMERCIO DE...,2333.902100
1682,NELCAST LIMITED,2276.036377
28,AAM INTERNATIONAL S A R L,2243.530762
126,ALUCROM Sp zo o,1634.161011
2253,STACKPOLE INTERNATIONAL FLUID POWER SOLUTIONS ...,1542.144287
2628,WHB AUTOMOTIVE S/A,1486.445312
1447,MAHINDRA CIE AUTOMOTIVE LTD,1355.732910
1880,PTI ASSEMBLY AND MACHINING LLC,1204.864868
1202,JIANGSU SHUANGHUAN GEAR CO LTD,1118.186157
1650,"Metaldyne Powertrain Components, Inc",982.959167


In [73]:
# selected variables previous model

In [ ]:
# selected variables new model

In [ ]:
kk = upload_df.sort_values('PRED_PREM_FREIGHT', ascending=False).head(25)
kk

In [ ]:
# ll[['VENDOR_NAME','PRED_PREM_FREIGHT_final','PRED_PREM_FREIGHT']]

### Delete older entries from SQL

In [ ]:
# tns_dsn = cx_Oracle.makedsn('aamlxbidbp001.aam.net', 1525, service_name='BIDWPRD')
# con_ORACLE = cx_Oracle.connect(user='PROD_AAM_RO', password='prodaamro', dsn=tns_dsn)


In [ ]:
# sql = ("delete from bi_dw.wc_sc_pred_s where MONTH = TO_DATE('"+str(cur_month)+"', 'YYYY-MM-DD' )")

# cursor = con_ORACLE.cursor()

# cursor.execute(sql)

# con_ORACLE.commit()
# cursor.close
# con_ORACLE.close  

### Save Output to Oracle

In [ ]:
# sql = ('insert into bi_dw.wc_sc_pred_s(VENDOR_NAME,MONTH,PRED_PRNR,PRED_PREM_FREIGHT,FLAG_PREM_FREIGHT,FLAG_PRNR, PRED_PPM, FLAG_PPM)'
#         'values(:VENDOR_NAME,:MONTH,:PRED_PRNR,:PRED_PREM_FREIGHT,:FLAG_PREM_FREIGHT,:FLAG_PRNR,:PRED_PPM, :FLAG_PPM)')

# cursor = con_ORACLE.cursor()

# upload_df = ll2[["VENDOR_NAME","MONTH","PRED_PRNR","PRED_PREM_FREIGHT","FLAG_PREM_FREIGHT","FLAG_PRNR","PRED_PPM","FLAG_PPM"]]
# upload_df.dropna(subset = ["VENDOR_NAME"], inplace = True)
# upload_df.fillna(0, inplace = True)

# df_list = upload_df.values.tolist()

# cursor.executemany(sql, df_list)

# con_ORACLE.commit()
# cursor.close
# con_ORACLE.close       

# Check Database 

In [ ]:
# backup_v1=pd.read_sql_query("select * from bi_dw.wc_sc_pred_s order by month desc", con_ORACLE)
# # backup.to_csv(os.getcwd()+r"\Data Files\backup_prod.csv", index=False)
# backup_v1.head()

In [ ]:
# backup_v1.MONTH.value_counts()

In [ ]:
# backup_v1.to_csv(os.getcwd()+r"\data\validate.csv", index=False)

In [ ]:
# upload_df